<div dir="rtl">

# 🟢 Vector Stores و Retrievers في LangChain

## ✅ مقدمة:
هذا الشرح يوضح لك فكرة **vector store** و **retriever** في LangChain.  
هما بيبقوا طبقات (abstractions) هدفها تسهيل استرجاع البيانات من قواعد بيانات vector أو أي مصدر خارجي، ودمجها في سير عمل الـ LLM (النموذج اللغوي).

💡 دا مهم جدًا في التطبيقات اللي بتحتاج تجيب بيانات وتربطها بالردود زي **retrieval-augmented generation (RAG)**.

---

## ✅ سنتعلم:
### 1️⃣ Documents (المستندات)
- هي وحدات البيانات النصية اللي بنخزنها في الـ vector store.
- كل Document بيبقى فيه نص (content) و metadata (زي العنوان، المصدر، الوقت).

### 2️⃣ Vector Stores (مخازن المتجهات)
- قواعد بيانات بتخزن الـ Documents بعد تحويلها لتمثيل عددي (vectors).
- الهدف منها: البحث السريع والتطابق في الفضاء المتجهي.
- أمثلة: **FAISS, Pinecone, Chroma, Weaviate**.

### 3️⃣ Retrievers (المسترجعات)
- أدوات وظيفتها **البحث عن الوثائق الأقرب** (top-K) بناءً على استعلام المستخدم.
- بتاخد سؤالك → بتحوله لـ vector → بتدور على الوثائق المشابهة → بترجع النتائج.
- بتساعد الموديل يجاوب بشكل مدعوم بمعلومات حقيقية (Grounded answers).

---

## ✅ الخلاصة:
> Vector Stores = مكان بنخزن فيه التمثيلات العددية للنصوص.  
> Retrievers = أداة بترجعلك النصوص الأقرب لاستفسارك.

</div>


In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
groq_api_key = os.getenv('GROQ_API_KEY')
from langchain_groq import ChatGroq

model = ChatGroq(model = 'Llama3-8b-8192', groq_api_key= groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002065B263B60>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002065B290C50>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

<div dir="rtl">

# 📄 Documents في LangChain

## ✅ ما هو الـ Document؟
في LangChain، الـ **Document** هو abstraction (طبقة تجريدية) بتمثل وحدة من النص ومعاها بيانات إضافية (metadata).

## ✅ يتكون من حاجتين رئيسيتين:
| الخاصية | الوصف |
|----------|-------|
| `page_content` | نص عادي بيمثل محتوى الوثيقة (string) |
| `metadata`     | قاموس (dict) فيه معلومات إضافية عن الوثيقة (زي المصدر، العلاقة بوثائق تانية... إلخ) |

🔹 الـ metadata ممكن تحتوي على:
- مصدر الوثيقة
- علاقاتها بوثائق تانية
- أي معلومات توضيحية إضافية

⚠️ ملحوظة:  
غالبًا الـ Document الواحد بيمثل **جزء (chunk)** من وثيقة أكبر.

---

In [2]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content='Dogs are great companions, know for their loyalty and friendliness.',
        metadata = {'source': 'mammal-pets-doc'}
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ), 
]

documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, know for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [ ]:
# Lets Embedding Documents
# HuggingFaceEmbeddings => on huggingface web in sentence transformer site
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\ahmed\anaconda3\envs\genenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
## Vectore Stores
from langchain_chroma import Chroma

# here we convert text to vectore and store in db
vectorstore=Chroma.from_documents(documents,embedding=embeddings)
vectorstore


In [7]:
vectorstore.similarity_search('cat')

[Document(id='2a46fc95-100c-4a45-937a-bf9a819d2b57', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='db00e989-5a5a-4249-baab-276221f7cfc9', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, know for their loyalty and friendliness.'),
 Document(id='67fcb40a-5f91-4bd7-b6d6-3f9305759b88', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='e7375d52-eb7c-4339-84e9-f419cabe8e41', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [8]:
vectorstore.similarity_search_with_score('cat')

[(Document(id='2a46fc95-100c-4a45-937a-bf9a819d2b57', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351057410240173),
 (Document(id='db00e989-5a5a-4249-baab-276221f7cfc9', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, know for their loyalty and friendliness.'),
  1.5318481922149658),
 (Document(id='67fcb40a-5f91-4bd7-b6d6-3f9305759b88', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  1.5956902503967285),
 (Document(id='e7375d52-eb7c-4339-84e9-f419cabe8e41', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  1.6657923460006714)]

<div dir="rtl">

# 🟢 asimilarity_search باختصار

- **asimilarity_search** = Asynchronous Similarity Search.
- نفس فكرة `similarity_search` (تدور على أقرب المستندات بالكلمات).
- الفرق:  
  ✅ **Async** → بتشتغل بـ `await` بدون تعطيل باقي الكود.

## 🕒 إمتى تستخدمها؟
- في التطبيقات السريعة (Web, API, Chatbots).
- عشان الكود يكمل شغله وانت مستني النتيجة.

In [10]:
await vectorstore.asimilarity_search('cat')

[Document(id='2a46fc95-100c-4a45-937a-bf9a819d2b57', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='db00e989-5a5a-4249-baab-276221f7cfc9', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, know for their loyalty and friendliness.'),
 Document(id='67fcb40a-5f91-4bd7-b6d6-3f9305759b88', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='e7375d52-eb7c-4339-84e9-f419cabe8e41', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

<div dir="rtl">

# 🟢 Retrievers في LangChain

## ✅ الفكرة ببساطة:
- **VectorStore** مش بيدعم **Runnable** (مش بيتوصل مباشر في LangChain Expression Language chains).
- **Retrievers** بيدعموا Runnable → يعني ينفع توصلهم في سلاسل LCEL بسهولة.

## ✅ الفرق الرئيسي:
| VectorStore | Retriever |
|-------------|-----------|
| بيعمل similarity_search | Runnable (ينفذ invoke, async, batch) |
| مش بيشتغل في LCEL مباشرة | مصمم يتوصل في LCEL chains |

## ✅ طيب ليه نستخدم Retriever؟
- بيدي **واجهة موحدة** للتعامل (sync/async/batch).
- بيسهل ربطه في **LangChain pipelines**.
- بيدعم أدوات تانية زي **contextual compression, filters, MMR...**.



<div dir="rtl">

# 🟢 يعني إيه عملية Retrieve؟

## ✅ الفكرة في كلمة:
**Retrieve = أجيب معلومة من الداتا اللي عندي (Vector Store) تكون شبه السؤال اللي جالي.**

## ✅ خطوات عملية الـ Retrieve ببساطة:
1. المستخدم بيسأل سؤال (Query).
2. نحول السؤال ده لــ **Vector** (أرقام تمثل معنى السؤال).
3. نقارن الــ Vector ده مع Vectors مخزنة عندي (Documents chunks).
4. نرجّع أقرب الحاجات اللي شبه السؤال (Similarity Search).
5. الموديل يستخدم النتيجة دي في الإجابة.

## ✅ مثال بسيط:
لو سألت:  
🗣️ "إيه هو الـ LangChain؟"  
هيجيب أقرب Document فيه تعريف LangChain ويرجعه.

## ✅ ملحوظة:
- VectorStore = مكان متخزن فيه المستندات كـ Vectors.
- Retriever = الكود اللي بيعمل العملية دي ويرجعلك الـ context المناسب.

</div>


In [14]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1) 
# k => هات دوك واحد بس اللى هو اقرب واحد
# هنا مستخدمين batch يعني هندخل كذا مره وكل مره ليها خرج لوحده
# فهو هيجيب اقرب دوك لكل كلمه على حدي
# ليه دوك واحد لكل واحدة علشان ال k = 1

retriever.batch(['cat', 'dog'])


[[Document(id='2a46fc95-100c-4a45-937a-bf9a819d2b57', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='db00e989-5a5a-4249-baab-276221f7cfc9', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, know for their loyalty and friendliness.')]]

<div dir="rtl">

# 🟢 VectorStore → Retriever (as_retriever)

## ✅ الفكرة:
- أي **VectorStore** عنده ميثود اسمها `as_retriever()`.
- بتطلعلك **Retriever** جاهز (نوعه: VectorStoreRetriever).

## ✅ ليه مهم؟
- عشان تربطه مباشرة مع LangChain chains كـ Runnable.
- بيحدد:
  - **search_type** → طريقة البحث (similarity / mmr / etc).
  - **search_kwargs** → إعدادات للبحث (زي top_k مثلاً).

In [ ]:
# نفس اللى عملناه فوق بس حولناه ل Retriever علشان نعرف ندخله ف Chain
retriever = vectorstore.as_retriever(
    search_type = 'similarity', # طريقة ايجاد الدوك الاقرب للى بندور عليه
    search_kwargs = {'k':1} # عايزه يرجع اقرب كام واحدة
)

retriever.batch(['Cat', 'Dog'])

[[Document(id='2a46fc95-100c-4a45-937a-bf9a819d2b57', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='db00e989-5a5a-4249-baab-276221f7cfc9', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, know for their loyalty and friendliness.')]]

# Rag Example

In [30]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer This Question using the provieded contect only.
{question}
context:
{context}
""" 

prompt = ChatPromptTemplate.from_messages([('human', message)])
rag_chain = {'context': retriever, 'question': RunnablePassthrough()}|prompt|model
response=rag_chain.invoke("tell me about dogs")
print(response.content)

According to the provided context, dogs are great companions, known for their loyalty and friendliness.


<div dir='rtl'>
- ده Pipeline (سلسلة) ماشية كالتالي:

- بياخد input (زي: "tell me about dogs").

- بيوديه للـ Retriever → يسترجع له context مناسب.

- بيعدي على RunnablePassthrough → ده بيعدي السؤال زي ما هو (ملوش شغل غير التمرير).

- بيملأ الـ Prompt بالقيم.

- بعدين بيروح للـ model يجاوب.